In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')

In [32]:
season = 2020
# csv to save
csv_file = '../' + 'Odds' + '/' + str(season) + '_odds_ds'+ '.csv'
print(csv_file)
odds_df = pd.read_csv(csv_file)

../Odds/2020_odds_ds.csv


In [33]:
odds_df.head(5)

,id,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,season,bookmaker,winner_home,winner_away,winner_draw,double_home_away,double_home_draw,double_away_draw,week_y
0,605124.0,533,532,6,-0.230920,1.042431,2.308306,1,1,140,2020,8,1.60,5.25,4.1,1.22,1.17,2.25,6.0
1,605117.0,543,548,6,0.029372,1.048234,2.271473,-1,1,140,2020,8,2.60,2.63,3.4,1.30,1.44,1.50,6.0
2,605119.0,545,727,6,1.182168,1.240975,1.048746,0,0,140,2020,8,2.70,2.90,3.0,1.36,1.40,1.44,6.0
3,605115.0,542,797,6,0.673380,2.285670,0.538878,-1,0,140,2020,8,1.75,5.50,3.3,1.33,1.14,2.00,6.0
4,605116.0,531,539,6,1.218447,1.058653,2.199194,1,1,140,2020,8,2.00,4.00,3.3,1.33,1.22,1.73,6.0


In [34]:
def get_max_favorite(row):
    if row["winner_home"] > row["winner_away"] and row["winner_home"] > row["winner_draw"]:
        return 1
    elif row["winner_away"] > row["winner_home"] and row["winner_away"] > row["winner_draw"]:
        return -1
    elif (row["winner_draw"] > row["winner_away"] and row["winner_draw"] > row["winner_home"]) or row["winner_home"] == row["winner_away"]:
        return 0

    # Expect to normalize probabilities by margin, by now returns the max probability
def get_favorite(df):
    fav = []
    for index, row in df.iterrows():
        fav.append(get_max_favorite(row))
    return fav

def prob(x):
    return (1/x)

# TODO: Resolve the normalized margin situation
def feature_engineer(odds):
    # Odds as probability (inverse prob)
    odds["winner_home"] = odds["winner_home"].apply(prob)
    odds["winner_away"] = odds["winner_away"].apply(prob)
    odds["winner_draw"] = odds["winner_draw"].apply(prob)
    odds["double_home_away"] = odds["double_home_away"].apply(prob)
    odds["double_home_draw"] = odds["double_home_draw"].apply(prob)
    odds["double_away_draw"] = odds["double_away_draw"].apply(prob)
    # Calculate margin for winner 1 - (sum of winner)
    odds["winner_margin"] = abs(1 - (odds["winner_home"] + odds["winner_away"] + odds["winner_draw"]))
    # Calculate winner (the favorite of the bookmaker)
    odds["favorite"] = get_favorite(odds)
    # If it is underdog (predict is different than the favorite)
    odds["underdog"] = (odds["favorite"] != odds["predict"]).astype(int)
    # Calculate margin for double --> TODO: try a normalization 
    # Each has a margin
    odds["double_margin_ha"] = odds["double_home_away"] - odds["winner_home"] - odds["winner_away"]
    odds["double_margin_hd"] = odds["double_home_draw"] - odds["winner_home"] - odds["winner_draw"]
    odds["double_margin_ad"] = odds["double_away_draw"] - odds["winner_away"] - odds["winner_draw"]
    # Calculate doble on predict
    odds["predict_home_away"] = odds["predict_win"] + odds["predict_lose"]
    odds["predict_home_draw"] = odds["predict_win"] + odds["predict_draw"]
    odds["predict_away_draw"] = odds["predict_lose"] + odds["predict_draw"]
    # Dropping bookmaker, season, id, week-y (is not completely full)
    odds = odds.drop(columns=['bookmaker', 'season', 'id', 'week_y'])
    return odds

In [35]:
transformed_odds = feature_engineer(odds_df)

In [36]:
transformed_odds.head(5)

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,winner_home,...,double_away_draw,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw
0,533,532,6,-0.230920,1.042431,2.308306,1,1,140,0.625000,...,0.444444,0.059379,1,0,0.004196,-0.014202,0.010066,2.077386,3.350737,0.811511
1,543,548,6,0.029372,1.048234,2.271473,-1,1,140,0.384615,...,0.666667,0.058961,1,0,0.004387,0.015711,-0.007679,2.300845,3.319706,1.077605
2,545,727,6,1.182168,1.240975,1.048746,0,0,140,0.370370,...,0.694444,0.048531,1,1,0.020096,0.010582,0.016284,2.230914,2.289721,2.423143
3,542,797,6,0.673380,2.285670,0.538878,-1,0,140,0.571429,...,0.500000,0.056277,1,1,-0.001367,0.002734,0.015152,1.212259,2.824548,2.959050
4,531,539,6,1.218447,1.058653,2.199194,1,1,140,0.500000,...,0.578035,0.053030,1,0,0.001880,0.016642,0.025004,3.417641,3.257847,2.277100


In [37]:
# Target the real
season = 2020
# csv to save
csv_file = '../' + 'Odds' + '/' + str(season) + '_' + '11_20_'+ 'odds_test'+ '.csv'
print(csv_file)
odds_test = pd.read_csv(csv_file)
transformed_odds_test = feature_engineer(odds_test)

../Odds/2020_11_20_odds_test.csv


In [38]:
transformed_odds_test.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,double_away_draw,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw
0,542,532,10,0.662601,2.258979,0.245079,0,140,0.380228,0.370370,...,0.694444,0.063099,1,1,-0.015304,0.001716,0.011574,0.907680,2.504058,2.921580
1,531,543,10,-0.233664,1.051158,2.308872,1,140,0.454545,0.303030,...,0.617284,0.060606,1,0,-0.005696,-0.005696,0.011223,2.075208,3.360030,0.817494
2,530,529,10,0.818050,1.247125,1.712625,0,140,0.344828,0.420168,...,0.714286,0.050710,-1,1,0.010198,0.023053,0.008403,2.530676,2.959750,2.065175
3,545,546,10,-0.224020,1.079172,2.235509,1,140,0.285714,0.434783,...,0.775194,0.065324,-1,1,-0.006211,0.006401,-0.004416,2.011489,3.314681,0.855152
4,724,548,10,1.204553,1.324190,2.194885,1,140,0.238095,0.546448,...,0.819672,0.062321,-1,1,-0.009350,0.030575,-0.004554,3.399438,3.519075,2.528743


In [39]:
transformed_odds_test.shape

(66, 23)

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn import preprocessing
def prediction(train, test):
    nb_ovo=OneVsOneClassifier(GaussianNB())
    #nb_ovo=OneVsOneClassifier(RandomForestClassifier(random_state=0, criterion="entropy", max_features="log2", min_samples_leaf=2))
    #nb_ovo=OneVsOneClassifier(SVC(kernel='linear', probability=True))
    target = train['real']
    predict = test['predict']
    favorite = test['favorite']
    train = train.drop(columns=['real'])
    #train = train.drop(columns=['real', 'predict', 'favorite', 'underdog'])
    #test = test.drop(columns=['predict', 'favorite', 'underdog'])
    train_set = preprocessing.StandardScaler().fit_transform(train)
    test_set = preprocessing.StandardScaler().fit_transform(test)
    nb_ovo.fit(train_set, target)
    Y = nb_ovo.predict(test_set)
    test['real'] = Y
    test['predict'] = predict
    test['favorite'] = favorite
    return test

In [11]:
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression

def evaluation(data, method, cross_val=5, k_best=23):
    if method == '1':
        target = data['real']
    elif method == '2':
        target = (data['real'] == data['favorite']).astype(int)
        train = data.drop(columns=['favorite', 'predict', 'underdog'])
    elif method == '3':
        target = (data['real'] == data['predict']).astype(int)
        train = data.drop(columns=['favorite', 'predict', 'underdog'])
    train = data.drop(columns=['real'])
    scaler = preprocessing.StandardScaler()
    feature_selection = SelectKBest(f_regression, k=k_best)
    if method == '1':
        nb_ovo=OneVsOneClassifier(GaussianNB())
    else:
        nb_ovo=SVC(kernel='linear', probability=True)
    pipe = Pipeline([('scaler', scaler), ("anova", feature_selection), ('nb', nb_ovo)])
    param_grid = {
    }
    grid = GridSearchCV(pipe, param_grid, cv=cross_val, refit=True)
    grid.fit(train, target)
    y_pred = cross_val_predict(grid, train, target, cv=cross_val)
    print("Confusion Matrix:\n", confusion_matrix(target, y_pred))
    print("Classification Report:\n", classification_report(target, y_pred))

In [24]:
def prediction_winner(data, test_data, method, k_best=23):
    if method == '1':
        target = data['real']
    elif method == '2':
        target = (data['real'] == data['favorite']).astype(int)
        train = data.drop(columns=['favorite', 'predict', 'underdog'])
        test = test_data.drop(columns=['favorite', 'predict', 'underdog'])
    elif method == '3':
        target = (data['real'] == data['predict']).astype(int)
        train = data.drop(columns=['favorite', 'predict', 'underdog'])
        test = test_data.drop(columns=['favorite', 'predict', 'underdog'])
    train = data.drop(columns=['real'])
    test = test_data.drop(columns=['real'])
    scaler = preprocessing.StandardScaler()
    feature_selection = SelectKBest(f_regression, k=k_best)
    if method == '1':
        nb_ovo=OneVsOneClassifier(GaussianNB())
    else:
        nb_ovo=SVC(kernel='linear', probability=True)
    pipe = Pipeline([('scaler', scaler), ("anova", feature_selection), ('nb', nb_ovo)])
    param_grid = {
    }
    grid = GridSearchCV(pipe, param_grid, refit=True)
    grid.fit(train, target)
    y_pred = grid.predict(test)
    test_data["invest"] = y_pred
    return test_data

In [41]:
investment = prediction_winner(transformed_odds, new_prediction, method='2')
investment.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw,real,invest
0,542,532,10,0.662601,2.258979,0.245079,0,140,0.380228,0.370370,...,1,1,-0.015304,0.001716,0.011574,0.907680,2.504058,2.921580,0,0
1,531,543,10,-0.233664,1.051158,2.308872,1,140,0.454545,0.303030,...,1,0,-0.005696,-0.005696,0.011223,2.075208,3.360030,0.817494,0,0
2,530,529,10,0.818050,1.247125,1.712625,0,140,0.344828,0.420168,...,-1,1,0.010198,0.023053,0.008403,2.530676,2.959750,2.065175,0,0
3,545,546,10,-0.224020,1.079172,2.235509,1,140,0.285714,0.434783,...,-1,1,-0.006211,0.006401,-0.004416,2.011489,3.314681,0.855152,0,0
4,724,548,10,1.204553,1.324190,2.194885,1,140,0.238095,0.546448,...,-1,1,-0.009350,0.030575,-0.004554,3.399438,3.519075,2.528743,-1,0


In [30]:
df_league = investment[(investment["league"]==88)]
df_league = df_league[["home_team.id", "away_team.id", "predict", "favorite", "real", "invest"]]
df_league.head(10)

,home_team.id,away_team.id,predict,favorite,real,invest
49,195,204,1,1,1,1
50,201,208,1,1,1,1
51,193,207,-1,-1,-1,1
52,426,198,1,1,0,0
53,205,209,-1,-1,-1,1
54,202,200,-1,-1,-1,1
55,415,197,-1,-1,-1,1
56,417,210,-1,-1,-1,1


In [42]:
df_league = investment[(investment["league"]==88)]
df_league = df_league[["home_team.id", "away_team.id", "predict", "favorite", "real", "invest"]]
df_league.head(10)

,home_team.id,away_team.id,predict,favorite,real,invest
49,195,204,1,1,1,1
50,201,208,1,1,1,1
51,193,207,-1,-1,-1,1
52,426,198,1,1,0,0
53,205,209,-1,-1,-1,1
54,202,200,-1,-1,-1,1
55,415,197,-1,-1,-1,1
56,417,210,-1,-1,-1,1


In [40]:
new_prediction = prediction(transformed_odds, transformed_odds_test)

In [13]:
new_prediction.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw,real
0,542,532,10,0.662601,2.258979,0.245079,0,140,0.380228,0.370370,...,0.063099,1,1,-0.015304,0.001716,0.011574,0.907680,2.504058,2.921580,0
1,531,543,10,-0.233664,1.051158,2.308872,1,140,0.454545,0.303030,...,0.060606,1,0,-0.005696,-0.005696,0.011223,2.075208,3.360030,0.817494,0
2,530,529,10,0.818050,1.247125,1.712625,0,140,0.344828,0.420168,...,0.050710,-1,1,0.010198,0.023053,0.008403,2.530676,2.959750,2.065175,0
3,545,546,10,-0.224020,1.079172,2.235509,1,140,0.285714,0.434783,...,0.065324,-1,1,-0.006211,0.006401,-0.004416,2.011489,3.314681,0.855152,0
4,724,548,10,1.204553,1.324190,2.194885,1,140,0.238095,0.546448,...,0.062321,-1,1,-0.009350,0.030575,-0.004554,3.399438,3.519075,2.528743,-1


In [14]:
new_prediction.shape

(66, 24)

In [20]:
df_league = new_prediction[(new_prediction["league"]==88)]
df_league = df_league[["home_team.id", "away_team.id", "predict", "favorite", "real"]]
df_league.head(10)

,home_team.id,away_team.id,predict,favorite,real
49,195,204,1,1,1
50,201,208,1,1,1
51,193,207,-1,-1,-1
52,426,198,1,1,0
53,205,209,-1,-1,-1
54,202,200,-1,-1,-1
55,415,197,-1,-1,-1
56,417,210,-1,-1,-1


In [45]:
df_league = new_prediction[(new_prediction["league"]==140)]
df_league.head(10)

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,predict,league,winner_home,winner_away,...,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw,real,invest
0,542,532,10,0.662601,2.258979,0.245079,0,140,0.380228,0.370370,...,1,1,-0.015304,0.001716,0.011574,0.907680,2.504058,2.921580,0,0
1,531,543,10,-0.233664,1.051158,2.308872,1,140,0.454545,0.303030,...,1,0,-0.005696,-0.005696,0.011223,2.075208,3.360030,0.817494,0,0
2,530,529,10,0.818050,1.247125,1.712625,0,140,0.344828,0.420168,...,-1,1,0.010198,0.023053,0.008403,2.530676,2.959750,2.065175,0,0
3,545,546,10,-0.224020,1.079172,2.235509,1,140,0.285714,0.434783,...,-1,1,-0.006211,0.006401,-0.004416,2.011489,3.314681,0.855152,0,0
4,724,548,10,1.204553,1.324190,2.194885,1,140,0.238095,0.546448,...,-1,1,-0.009350,0.030575,-0.004554,3.399438,3.519075,2.528743,-1,0
5,715,720,10,-0.229651,1.011041,2.309839,1,140,0.487805,0.263158,...,1,0,-0.015669,-0.000305,0.002377,2.080188,3.320880,0.781390,1,0
6,539,797,10,0.932566,1.334443,2.202957,1,140,0.617284,0.190476,...,1,0,0.011912,-0.012583,0.014069,3.135523,3.537401,2.267009,0,1
7,727,726,10,0.034580,1.055454,2.250405,1,140,0.400000,0.322581,...,1,0,0.012713,0.001961,0.010753,2.284984,3.305859,1.090034,0,0
8,536,538,10,1.205339,1.223703,0.788635,0,140,0.578035,0.190476,...,1,1,0.006683,-0.001112,0.031746,1.993974,2.012338,2.429042,0,0
9,533,541,10,2.308729,0.513096,0.793649,-1,140,0.363636,0.434783,...,-1,0,0.001581,0.016896,0.023093,3.102378,1.306745,2.821825,0,1


In [99]:
"""
    Evaluate the new prediction
    - Combination: win, draw, lose
    - As a favorite: winner or not winner
    - As a prediction: winner or not winner
"""

'\n    Evaluate the new prediction\n    - Combination: win, draw, lose\n    - As a favorite: winner or not winner\n    - As a prediction: winner or not winner\n'

In [15]:
# Combination win, draw, lose
evaluation(transformed_odds, method='1', k_best=14)

Confusion Matrix:
 [[56 63 14]
 [27 56 20]
 [18 69 58]]
Classification Report:
               precision    recall  f1-score   support

          -1       0.55      0.42      0.48       133
           0       0.30      0.54      0.38       103
           1       0.63      0.40      0.49       145

    accuracy                           0.45       381
   macro avg       0.49      0.45      0.45       381
weighted avg       0.51      0.45      0.46       381



In [16]:
# Is the favorite going to win?
evaluation(transformed_odds, method='2', k_best=23)

Confusion Matrix:
 [[128  62]
 [ 91 100]]
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.67      0.63       190
           1       0.62      0.52      0.57       191

    accuracy                           0.60       381
   macro avg       0.60      0.60      0.60       381
weighted avg       0.60      0.60      0.60       381



In [17]:
# Is the prediction going to win?
evaluation(transformed_odds, method='3', k_best=23)

Confusion Matrix:
 [[156  59]
 [ 77  89]]
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.73      0.70       215
           1       0.60      0.54      0.57       166

    accuracy                           0.64       381
   macro avg       0.64      0.63      0.63       381
weighted avg       0.64      0.64      0.64       381



In [18]:
# Combination: win, draw, lose
transformed_odds

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,winner_home,...,double_away_draw,winner_margin,favorite,underdog,double_margin_ha,double_margin_hd,double_margin_ad,predict_home_away,predict_home_draw,predict_away_draw
0,533,532,6,-0.230920,1.042431,2.308306,1,1,140,0.625000,...,0.444444,0.059379,1,0,0.004196,-0.014202,0.010066,2.077386,3.350737,0.811511
1,543,548,6,0.029372,1.048234,2.271473,-1,1,140,0.384615,...,0.666667,0.058961,1,0,0.004387,0.015711,-0.007679,2.300845,3.319706,1.077605
2,545,727,6,1.182168,1.240975,1.048746,0,0,140,0.370370,...,0.694444,0.048531,1,1,0.020096,0.010582,0.016284,2.230914,2.289721,2.423143
3,542,797,6,0.673380,2.285670,0.538878,-1,0,140,0.571429,...,0.500000,0.056277,1,1,-0.001367,0.002734,0.015152,1.212259,2.824548,2.959050
4,531,539,6,1.218447,1.058653,2.199194,1,1,140,0.500000,...,0.578035,0.053030,1,0,0.001880,0.016642,0.025004,3.417641,3.257847,2.277100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,741,600,12,1.844933,-0.072929,1.178289,-1,-1,144,0.512821,...,0.555556,0.065201,1,1,0.001465,-0.004293,0.003175,3.023222,1.105360,1.772004
377,734,263,12,1.844933,-0.072929,1.178289,0,-1,144,0.420168,...,0.653595,0.061582,1,1,-0.008611,0.016340,0.012181,3.023222,1.105360,1.772004
378,735,742,12,-0.201422,0.975275,2.205475,-1,1,144,0.285714,...,0.775194,0.071429,-1,1,-0.010520,0.006606,-0.010520,2.004054,3.180751,0.773854
379,624,569,12,1.844933,-0.072929,1.178289,-1,-1,144,0.230947,...,0.833333,0.069182,-1,0,0.000490,-0.004756,-0.004902,3.023222,1.105360,1.772004


In [44]:
print((47+64)/len(transformed_odds))
transformed_odds.groupby(['real', 'favorite']).count()

0.47844827586206895


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real favorite                                                                   
-1   -1.0                47            47      47            47            47   
      1.0                36            36      36            36            36   
 0   -1.0                24            24      24            24            24   
      1.0                40            40      40            40            40   
 1   -1.0                20            20      20            20            20   
      1.0                64            64      64            64            64   

               predict_win  predict  league  winner_home  winner_away  \
real favorite                                                           
-1   -1.0               47       47      47           47           47   
      1.0               36       36      36           36           36   
 0   -1.0               24       24      24           24           24   
      1.0               40       40      40           40           40   
 1   -1.0               20       20      20           20           20   
      1.0               64       64      64           64           64   

               winner_draw  double_home_away  double_home_draw  \
real favorite                                                    
-1   -1.0               47                47                47   
      1.0               36                36                36   
 0   -1.0               24                24                24   
      1.0               40                40                40   
 1   -1.0               20                20                20   
      1.0               64                64                64   

               double_away_draw  winner_margin  underdog  double_margin  \
real favorite                                                             
-1   -1.0                    47             47        47             47   
      1.0                    36             36        36             36   
 0   -1.0                    24             24        24             24   
      1.0                    40             40        40             40   
 1   -1.0                    20             20        20             20   
      1.0                    64             64        64             64   

               predict_home_away  predict_home_draw  predict_away_draw  
real favorite                                                           
-1   -1.0                     47                 47                 47  
      1.0                     36                 36                 36  
 0   -1.0                     24                 24                 24  
      1.0                     40                 40                 40  
 1   -1.0                     20                 20                 20  
      1.0                     64                 64                 64

In [43]:
print((33+10+53)/len(transformed_odds))
transformed_odds.groupby(['real', 'predict']).count()

0.41379310344827586


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real predict                                                                   
-1   -1                 33            33      33            33            33   
      0                 13            13      13            13            13   
      1                 38            38      38            38            38   
 0   -1                 20            20      20            20            20   
      0                 10            10      10            10            10   
      1                 34            34      34            34            34   
 1   -1                 20            20      20            20            20   
      0                 11            11      11            11            11   
      1                 53            53      53            53            53   

              predict_win  league  winner_home  winner_away  winner_draw  \
real predict                                                               
-1   -1                33      33           33           33           33   
      0                13      13           13           13           13   
      1                38      38           38           38           38   
 0   -1                20      20           20           20           20   
      0                10      10           10           10           10   
      1                34      34           34           34           34   
 1   -1                20      20           20           20           20   
      0                11      11           11           11           11   
      1                53      53           53           53           53   

              double_home_away  double_home_draw  double_away_draw  \
real predict                                                         
-1   -1                     33                33                33   
      0                     13                13                13   
      1                     38                38                38   
 0   -1                     20                20                20   
      0                     10                10                10   
      1                     34                34                34   
 1   -1                     20                20                20   
      0                     11                11                11   
      1                     53                53                53   

              winner_margin  favorite  underdog  double_margin  \
real predict                                                     
-1   -1                  33        32        33             33   
      0                  13        13        13             13   
      1                  38        38        38             38   
 0   -1                  20        20        20             20   
      0                  10        10        10             10   
      1                  34        34        34             34   
 1   -1                  20        20        20             20   
      0                  11        11        11             11   
      1                  53        53        53             53   

              predict_home_away  predict_home_draw  predict_away_draw  
real predict                                                           
-1   -1                      33                 33                 33  
      0                      13                 13                 13  
      1                      38                 38                 38  
 0   -1                      20                 20                 20  
      0                      10                 10                 10  
      1                      34                 34                 34  
 1   -1                      20                 20                 20  
      0                      11                 11                 11  
      1                      53                 53                 53

In [23]:
# How many underdogs i found?
underdogs = transformed_odds[(transformed_odds["underdog"] == 1)]
underdogs.head()

,home_team.id,away_team.id,week_x,predict_lose,predict_draw,predict_win,real,predict,league,winner_home,...,double_home_away,double_home_draw,double_away_draw,winner_margin,favorite,underdog,double_margin,predict_home_away,predict_home_draw,predict_away_draw
2,545,727,6,1.182168,1.240975,1.048746,0,0,140,0.370370,...,0.735294,0.714286,0.694444,-0.048531,1.0,1,-1.144024,2.230914,2.289721,2.423143
3,542,797,6,0.673380,2.285670,0.538878,-1,0,140,0.571429,...,0.751880,0.877193,0.500000,-0.056277,1.0,1,-1.129073,1.212259,2.824548,2.959050
7,724,533,7,0.554033,0.520371,2.293519,0,1,140,0.266667,...,0.751880,0.578035,0.800000,-0.057502,-1.0,1,-1.129914,2.847552,2.813889,1.074404
8,529,541,7,2.244724,0.509370,0.788361,-1,-1,140,0.512821,...,0.775194,0.800000,0.555556,-0.053756,1.0,1,-1.130749,3.033085,1.297731,2.754094
11,536,545,7,0.968180,1.230500,1.034634,-1,0,140,0.689655,...,0.833333,0.909091,0.380228,-0.063459,1.0,1,-1.122652,2.002814,2.265134,2.198680


In [24]:
underdogs.shape

(99, 22)

In [31]:
# How many times the underdog won?
print(44/99)
underdogs.groupby(['real', 'favorite']).count()

0.4444444444444444


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real favorite                                                                   
-1   -1.0                24            24      24            24            24   
      1.0                16            16      16            16            16   
 0   -1.0                10            10      10            10            10   
      1.0                15            15      15            15            15   
 1   -1.0                13            13      13            13            13   
      1.0                20            20      20            20            20   

               predict_win  predict  league  winner_home  winner_away  \
real favorite                                                           
-1   -1.0               24       24      24           24           24   
      1.0               16       16      16           16           16   
 0   -1.0               10       10      10           10           10   
      1.0               15       15      15           15           15   
 1   -1.0               13       13      13           13           13   
      1.0               20       20      20           20           20   

               winner_draw  double_home_away  double_home_draw  \
real favorite                                                    
-1   -1.0               24                24                24   
      1.0               16                16                16   
 0   -1.0               10                10                10   
      1.0               15                15                15   
 1   -1.0               13                13                13   
      1.0               20                20                20   

               double_away_draw  winner_margin  underdog  double_margin  \
real favorite                                                             
-1   -1.0                    24             24        24             24   
      1.0                    16             16        16             16   
 0   -1.0                    10             10        10             10   
      1.0                    15             15        15             15   
 1   -1.0                    13             13        13             13   
      1.0                    20             20        20             20   

               predict_home_away  predict_home_draw  predict_away_draw  
real favorite                                                           
-1   -1.0                     24                 24                 24  
      1.0                     16                 16                 16  
 0   -1.0                     10                 10                 10  
      1.0                     15                 15                 15  
 1   -1.0                     13                 13                 13  
      1.0                     20                 20                 20

In [32]:
# How many times the underdog won?
print(39/99)
underdogs.groupby(['real', 'predict']).count()

0.3939393939393939


home_team.id  away_team.id  week_x  predict_lose  predict_draw  \
real predict                                                                   
-1   -1                 10            10      10            10            10   
      0                 13            13      13            13            13   
      1                 18            18      18            18            18   
 0   -1                  6             6       6             6             6   
      0                 10            10      10            10            10   
      1                  9             9       9             9             9   
 1   -1                 13            13      13            13            13   
      0                 11            11      11            11            11   
      1                  9             9       9             9             9   

              predict_win  league  winner_home  winner_away  winner_draw  \
real predict                                                               
-1   -1                10      10           10           10           10   
      0                13      13           13           13           13   
      1                18      18           18           18           18   
 0   -1                 6       6            6            6            6   
      0                10      10           10           10           10   
      1                 9       9            9            9            9   
 1   -1                13      13           13           13           13   
      0                11      11           11           11           11   
      1                 9       9            9            9            9   

              double_home_away  double_home_draw  double_away_draw  \
real predict                                                         
-1   -1                     10                10                10   
      0                     13                13                13   
      1                     18                18                18   
 0   -1                      6                 6                 6   
      0                     10                10                10   
      1                      9                 9                 9   
 1   -1                     13                13                13   
      0                     11                11                11   
      1                      9                 9                 9   

              winner_margin  favorite  underdog  double_margin  \
real predict                                                     
-1   -1                  10         9        10             10   
      0                  13        13        13             13   
      1                  18        18        18             18   
 0   -1                   6         6         6              6   
      0                  10        10        10             10   
      1                   9         9         9              9   
 1   -1                  13        13        13             13   
      0                  11        11        11             11   
      1                   9         9         9              9   

              predict_home_away  predict_home_draw  predict_away_draw  
real predict                                                           
-1   -1                      10                 10                 10  
      0                      13                 13                 13  
      1                      18                 18                 18  
 0   -1                       6                  6                  6  
      0                      10                 10                 10  
      1                       9                  9                  9  
 1   -1                      13                 13                 13  
      0                      11                 11                 11  
      1                       9                  9                  9

In [ ]:
# If this was efficient? wins underdog / Underdogs detected

In [ ]:
# Compare winner vs real (odds favorite) and predict vs real (footballmlp fav)